In [107]:
from tensorflow.keras import datasets
from PIL import Image
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.backend import set_session as K
from sklearn.model_selection import cross_val_score
from tensorflow.keras.optimizers import SGD, Adam
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import matplotlib.pyplot as plt
import numpy as np
import glob

In [108]:
train_dir = './Animal Classification/Training Data' # train_data 경로
test_dir = './Animal Classification/Testing Data'   # test_data 경로
categories = ['Cat','Cow','Dog','Elephant','Panda'] # 동물 종류

# ImageDataGenerator 생성
train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

# ImageDataGenerator 설정
train_generator = train_datagen.flow_from_directory(
    train_dir, #학습용 이미지를 가져올 폴더명
    classes= categories,              # Cat 폴더의 image는 label을 0으로
                                      # Cow 폴더의 image는 label을 1로 설정한다
    target_size=(100,100),            # 이미지를 (64, 64)를 resize
    batch_size=20,                    # 한번에 10개의 이미지만 가져와요
    class_mode='categorical'              # 다중분류인경우 설정
)

# ImageDataGenerator 설정
test_generator = test_datagen.flow_from_directory(
    test_dir,
    classes=categories,         
                                      
    target_size=(100,100),            
    batch_size=20,                    
    class_mode='categorical'               
)

Found 7500 images belonging to 5 classes.
Found 3700 images belonging to 5 classes.


In [124]:
# CNN Model 구현
model = Sequential()
 
model.add(Conv2D(filters=32,                          # 32 * 3 *(3 * 3) + 32(bias) = 896
                 kernel_size=(3,3),
                 strides=(1,1),
                 activation='relu',
                 input_shape=(100,100,3)))

model.add(Conv2D(filters=64,                         # 32 * 63 * (3 * 3) + 64 = 18,496
                 kernel_size=(3,3),
                 strides=(1,1),
                 activation='relu'))
model.add(Dropout(0.8))                             # 드롭아웃 추가. 비율은 50%

model.add(MaxPooling2D(pool_size=(2,2)))                           
model.add(Dropout(0.8))            

model.add(Conv2D(filters=64,                       # 64 * 64 * (3 * 3) + 64 = 36,928
                 kernel_size=(3,3),
                 strides=(1,1),
                 activation='relu'))

#Fully Connection Layer(CNN)의 input_layer
model.add(Flatten())

# hidden Layer
model.add(Dense(units=128,
                activation='relu'))
# output_layer
model.add(Dense(units=5,
                activation='softmax'))


In [125]:
model.summary()

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_82 (Conv2D)           (None, 98, 98, 32)        896       
_________________________________________________________________
conv2d_83 (Conv2D)           (None, 96, 96, 64)        18496     
_________________________________________________________________
dropout_16 (Dropout)         (None, 96, 96, 64)        0         
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 48, 48, 64)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_84 (Conv2D)           (None, 46, 46, 64)        36928     
_________________________________________________________________
flatten_26 (Flatten)         (None, 135424)          

In [126]:
#model이 어떻게 도착하는지를 지정
model.compile(optimizer=Adam(learning_rate=0.001),
             loss="categorical_crossentropy",
             metrics=['accuracy']
             )

In [ ]:
hist = model.fit(train_generator,
                steps_per_epoch=375,
                epochs=20,
                verbose=1,
                validation_data=test_generator,
                validation_steps=60)

Epoch 1/20
375/375 [==============================] - 65s 173ms/step - loss: 1.9073 - accuracy: 0.1948 - val_loss: 1.6101 - val_accuracy: 0.1967
Epoch 2/20
375/375 [==============================] - 133s 355ms/step - loss: 1.6097 - accuracy: 0.1948 - val_loss: 1.6093 - val_accuracy: 0.2517
Epoch 3/20
375/375 [==============================] - 72s 193ms/step - loss: 1.6091 - accuracy: 0.1984 - val_loss: 1.6035 - val_accuracy: 0.2225
Epoch 4/20
375/375 [==============================] - 77s 206ms/step - loss: 1.5079 - accuracy: 0.3492 - val_loss: 1.6371 - val_accuracy: 0.2258
Epoch 5/20
206/375 [===============>..............] - ETA: 19s - loss: 1.3332 - accuracy: 0.4617